In [1]:
# !pip install psycopg2
# !wget -P /content https://jdbc.postgresql.org/download/postgresql-42.2.27.jar
# jdbc_driver_path = "/content/postgresql-42.2.27.jar"
postgres_url = 'jdbc:postgresql://localhost:5432/postgres'
jdbc_driver_path = '/opt/spark/jars/postgresql-42.3.1.jar'


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.jars", jdbc_driver_path) \
    .appName("WeatherAPIandETL") \
    .getOrCreate()



25/03/15 16:05:01 WARN Utils: Your hostname, tanvi-linux resolves to a loopback address: 127.0.1.1; using 192.168.1.74 instead (on interface wlo1)
25/03/15 16:05:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/15 16:05:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from datetime import datetime

print(datetime.now())
spark.sparkContext
spark.version

2025-03-15 16:05:10.432617


'3.4.4'

In [ ]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
# import os
# print(os.listdir())

In [ ]:
# import json

# with open("/home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json", "r") as opfile:
#     data = json.load(opfile)

# print(data)

# with open("/home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_new.json", "w") as f:
#     for record in data:
#         f.write(json.dumps(record) + "\n")  # ✅ Writes each object on a new line.


In [ ]:
# df = spark.read.option("multiLine", True).json("/content/weather_20250224.json")

In [ ]:
# df.printSchema()
# df.show()

In [ ]:
# !ls -ltr /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_deindent.json
#!chmod 777 /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json
#cat /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json | python -m json.tools

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *  # Optional: to use SQL functions like col(), lit(), etc.

# df.show()

# df.createOrReplaceTempView("weather_json")
# query = "SELECT * FROM weather_json"
# result = spark.sql(query)


In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import get_json_object, col

# result = df.select( "location"
    # get_json_object("location", "$.name").alias("name")
    # )

# result.show(truncate=False)

# newdf = df.select("location", "current")

# result.show(5)

In [ ]:

# data = [
#     ("1", """{"name": "John", "age": 30, "address": {"city": "New York", "zip": "10001"}, "phones": [{"type": "home", "number": "212-555-1234"}, {"type": "work", "number": "646-555-5678"}]}"""),
#     ("2", """{"name": "Jane", "age": 25, "address": {"city": "Boston", "zip": "02108"}, "phones": [{"type": "mobile", "number": "617-555-4321"}]}""")
# ]

# samdf = spark.createDataFrame(data, ["id", "json_data"])

# # Extract various fields
# result = samdf.select(
#     "id",
#     get_json_object("json_data", "$.name").alias("name"),
#     get_json_object("json_data", "$.age").alias("age"),
#     get_json_object("json_data", "$.address.city").alias("city"),
#     get_json_object("json_data", "$.phones[0].number").alias("first_phone")
# )

# # result.show(truncate=False)
# samdf.show()

In [4]:
import requests
api_dict = {
"API_KEY": "c2fe5ebd74734989a33163320252302",
"BASE_URL": "http://api.weatherapi.com/v1/forecast.json",
}

params= {"key":api_dict["API_KEY"], "q": "08854", "days": "1", "aqi": "no", "alerts": "no"}
response = requests.get(api_dict["BASE_URL"], params=params)
weather_data = response.json()
print(weather_data)

{'location': {'name': 'Piscataway', 'region': 'New Jersey', 'country': 'USA', 'lat': 40.5514984130859, 'lon': -74.4589996337891, 'tz_id': 'America/New_York', 'localtime_epoch': 1742069043, 'localtime': '2025-03-15 16:04'}, 'current': {'last_updated_epoch': 1742068800, 'last_updated': '2025-03-15 16:00', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 82, 'cloud': 100, 'feelslike_c': 8.8, 'feelslike_f': 47.8, 'windchill_c': 6.4, 'windchill_f': 43.5, 'heatindex_c': 8.0, 'heatindex_f': 46.5, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.9, 'gust_mph': 9.2, 'gust_kph': 14.8}, 'forecast': {'forecastday': [{'date': '2025-03-15', 'date_epoch': 1741996800, 'day': {'maxtemp_c': 8.3, 'maxtemp_f':

In [8]:
from json import dumps, dump
from pathlib import Path
base = "/home/tanvi/Documents/GitRepo/automatic-system/etl"
json_file_path = Path(base).absolute().joinpath("data", "weather", f"weather_{datetime.today().strftime('%Y%m%d')}.json")

with open(json_file_path, "w") as writej:
    # writej.write(dumps(weather_data, indent=2))
    dump(weather_data, writej)

In [9]:
postgres_url = 'jdbc:postgresql://localhost:5432/postgres'

postgres_properties = {
    'user': 'postgres',
    'password': 'myPassword',
    'driver': 'org.postgresql.Driver'}


In [10]:
import uuid

single_uuid = uuid.uuid4()

In [23]:
# print(type(json_file_path.absolute()),json_file_path.absolute())

jdf = spark.read.json(str(json_file_path))

# jdf = spark.read.json(weather_data)

jdf.printSchema()
jdf.show(truncate=False)

root
 |-- current: struct (nullable = true)
 |    |-- cloud: long (nullable = true)
 |    |-- condition: struct (nullable = true)
 |    |    |-- code: long (nullable = true)
 |    |    |-- icon: string (nullable = true)
 |    |    |-- text: string (nullable = true)
 |    |-- dewpoint_c: double (nullable = true)
 |    |-- dewpoint_f: double (nullable = true)
 |    |-- feelslike_c: double (nullable = true)
 |    |-- feelslike_f: double (nullable = true)
 |    |-- gust_kph: double (nullable = true)
 |    |-- gust_mph: double (nullable = true)
 |    |-- heatindex_c: double (nullable = true)
 |    |-- heatindex_f: double (nullable = true)
 |    |-- humidity: long (nullable = true)
 |    |-- is_day: long (nullable = true)
 |    |-- last_updated: string (nullable = true)
 |    |-- last_updated_epoch: long (nullable = true)
 |    |-- precip_in: double (nullable = true)
 |    |-- precip_mm: double (nullable = true)
 |    |-- pressure_in: double (nullable = true)
 |    |-- pressure_mb: double (n

In [24]:
print(weather_data)

jdf.createOrReplaceTempView("wdata")
spark.sql("select * from wdata").show()

{'location': {'name': 'Piscataway', 'region': 'New Jersey', 'country': 'USA', 'lat': 40.5514984130859, 'lon': -74.4589996337891, 'tz_id': 'America/New_York', 'localtime_epoch': 1742069043, 'localtime': '2025-03-15 16:04'}, 'current': {'last_updated_epoch': 1742068800, 'last_updated': '2025-03-15 16:00', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 82, 'cloud': 100, 'feelslike_c': 8.8, 'feelslike_f': 47.8, 'windchill_c': 6.4, 'windchill_f': 43.5, 'heatindex_c': 8.0, 'heatindex_f': 46.5, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.9, 'gust_mph': 9.2, 'gust_kph': 14.8}, 'forecast': {'forecastday': [{'date': '2025-03-15', 'date_epoch': 1741996800, 'day': {'maxtemp_c': 8.3, 'maxtemp_f':

Py4JError: An error occurred while calling o32.sql. Trace:
py4j.Py4JException: Method sql([class java.lang.String, class [Ljava.lang.Object;]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



In [ ]:
from pyspark.sql.functions import lit
location_list = [weather_data['location']]
df_l = spark.createDataFrame(location_list)
df_l = df_l.withColumn("process_uuid", lit(str(single_uuid)))
df_l = df_l.withColumn("zipcode", lit("08854"))
df_l.show()

In [ ]:
df_l = df_l.select("name","region","country","lat","lon","tz_id","localtime_epoch","localtime","process_uuid","zipcode")
df_l = df_l.withColumnRenamed("name", "city_name") \
           .withColumnRenamed("lat", "latitude") \
           .withColumnRenamed("lon", "longitude") \
           .withColumnRenamed("tz_id", "timezone_id") \
           .withColumnRenamed("localtime_epoch", "local_time_epoch") \
           .withColumnRenamed("localtime", "local_time")

df_l.write.jdbc(
    url=postgres_url,
    table="stage.location",
    mode="append",
    properties=postgres_properties
)

In [ ]:
from pyspark.sql.functions import col

currently_list = [weather_data['current']]
df_c = spark.createDataFrame(currently_list)
df_cf = df_c.selectExpr("last_updated_epoch","last_updated","temp_c","temp_f","is_day","wind_mph","wind_kph","wind_degree","wind_dir","pressure_mb","pressure_in","precip_mm","precip_in","humidity","cloud","feelslike_c","feelslike_f","windchill_c","windchill_f","heatindex_c","heatindex_f","dewpoint_c","dewpoint_f","vis_km","vis_miles","uv","gust_mph","gust_kph","condition.text as condition_text","condition.icon as condition_icon","condition.code as condition_code")
df_cf = df_cf.withColumn("process_uuid", lit(str(single_uuid)))
df_cf = df_cf.withColumn("zipcode", lit("08854"))
df_cf = df_cf.withColumn("last_updated", col("last_updated").cast("timestamp"))
df_cf.show()

In [ ]:
df_cf = df_cf.select("last_updated_epoch","last_updated","temp_c","temp_f","is_day","wind_mph","wind_kph","wind_degree","wind_dir","pressure_mb","pressure_in","precip_mm","precip_in","humidity","cloud","feelslike_c","feelslike_f","windchill_c","windchill_f","heatindex_c","heatindex_f","dewpoint_c","dewpoint_f","vis_km","vis_miles","uv","gust_mph","gust_kph","process_uuid","zipcode","condition_text","condition_icon","condition_code")
df_cf.write.jdbc(url=postgres_url, table="stage.currently",mode="append",properties=postgres_properties)

In [ ]:
daily_list = [weather_data['forecast']['forecastday'][0]['day']]
df_d = spark.createDataFrame(daily_list)
df_df = df_d.selectExpr("maxtemp_c","maxtemp_f","mintemp_c","mintemp_f","avgtemp_c","avgtemp_f","maxwind_mph","maxwind_kph","totalprecip_mm","totalprecip_in","totalsnow_cm","avgvis_km","avgvis_miles","avghumidity","daily_will_it_rain","daily_chance_of_rain","daily_will_it_snow","daily_chance_of_snow","uv","condition.text as condition_text","condition.icon as condition_icon","condition.code as condition_code")
df_df = df_df.withColumn("process_uuid", lit(str(single_uuid)))
df_df = df_df.withColumn("zipcode", lit("08854"))
df_df.show()

In [ ]:
df_df = df_df.select("maxtemp_c","maxtemp_f","mintemp_c","mintemp_f","avgtemp_c","avgtemp_f","maxwind_mph","maxwind_kph","totalprecip_mm","totalprecip_in","totalsnow_cm","avgvis_km","avgvis_miles","avghumidity","daily_will_it_rain","daily_chance_of_rain","daily_will_it_snow","daily_chance_of_snow","uv","process_uuid","zipcode","condition_text","condition_icon","condition_code")
df_df.write.jdbc(url=postgres_url,table="stage.daily",mode="append",properties=postgres_properties)

In [ ]:
from pyspark.sql.types import *

hourly_list = weather_data['forecast']['forecastday'][0]['hour']
date_var = weather_data['forecast']['forecastday'][0]['date']
epoch_var = weather_data['forecast']['forecastday'][0]['date_epoch']

schema = StructType([
    StructField("time_epoch", LongType(), True),
    StructField("time", StringType(), True),
    StructField("temp_c", DoubleType(), True),
    StructField("temp_f", DoubleType(), True),
    StructField("is_day", DoubleType(), True),
    StructField("wind_mph", DoubleType(), True),
    StructField("wind_kph", DoubleType(), True),
    StructField("wind_degree", DoubleType(), True),
    StructField("wind_dir", StringType(), True),
    StructField("pressure_mb", DoubleType(), True),
    StructField("pressure_in", DoubleType(), True),
    StructField("precip_mm", DoubleType(), True),
    StructField("precip_in", DoubleType(), True),
    StructField("snow_cm", DoubleType(), True),
    StructField("humidity", DoubleType(), True),
    StructField("cloud", DoubleType(), True),
    StructField("feelslike_c", DoubleType(), True),
    StructField("feelslike_f", DoubleType(), True),
    StructField("windchill_c", DoubleType(), True),
    StructField("windchill_f", DoubleType(), True),
    StructField("heatindex_c", DoubleType(), True),
    StructField("heatindex_f", DoubleType(), True),
    StructField("dewpoint_c", DoubleType(), True),
    StructField("dewpoint_f", DoubleType(), True),
    StructField("will_it_rain", DoubleType(), True),
    StructField("chance_of_rain", DoubleType(), True),
    StructField("will_it_snow", DoubleType(), True),
    StructField("chance_of_snow", DoubleType(), True),
    StructField("vis_km", DoubleType(), True),
    StructField("vis_miles", DoubleType(), True),
    StructField("gust_mph", DoubleType(), True),
    StructField("gust_kph", DoubleType(), True),
    StructField("uv", DoubleType(), True),
    StructField("condition", StructType([
        StructField("text", StringType(), True),
        StructField("icon", StringType(), True),
        StructField("code", StringType(), True)
    ]), True),
])
exclude_columns = {"time_epoch", "condition"}
hourly_list = [
    {key: float(value) if isinstance(value, int) and key not in exclude_columns else value 
     for key, value in row.items()}
    for row in hourly_list
]



In [ ]:
df_h = spark.createDataFrame(hourly_list, schema=schema)
df_hf = df_h.selectExpr("time_epoch","time","temp_c","temp_f","is_day","wind_mph","wind_kph","wind_degree","wind_dir","pressure_mb","pressure_in","precip_mm","precip_in","snow_cm","humidity","cloud","feelslike_c","feelslike_f","windchill_c","windchill_f","heatindex_c","heatindex_f","dewpoint_c","dewpoint_f","will_it_rain","chance_of_rain","will_it_snow","chance_of_snow","vis_km","vis_miles","gust_mph","gust_kph","uv","condition.text as condition_text","condition.icon as condition_icon","condition.code as condition_code")
df_hf = df_hf.withColumn("date", lit(date_var))
df_hf = df_hf.withColumn("date_epoch", lit(epoch_var))
df_hf = df_hf.withColumn("process_uuid", lit(str(single_uuid)))
df_hf = df_hf.withColumn("zipcode", lit("08854"))
df_hf = df_hf.withColumn("time", col("time").cast("timestamp"))
df_hf = df_hf.withColumn("date", col("date").cast("date"))
df_hf.show()

In [ ]:
df_hf = df_hf.select("time_epoch","time","temp_c","temp_f","is_day","wind_mph","wind_kph","wind_degree","wind_dir","pressure_mb","pressure_in","precip_mm","precip_in","snow_cm","humidity","cloud","feelslike_c","feelslike_f","windchill_c","windchill_f","heatindex_c","heatindex_f","dewpoint_c","dewpoint_f","will_it_rain","chance_of_rain","will_it_snow","chance_of_snow","vis_km","vis_miles","gust_mph","gust_kph","uv","condition_text","condition_icon","condition_code","date","date_epoch","process_uuid","zipcode")
df_hf.write.jdbc(url=postgres_url,table="stage.hourly",mode="append",properties=postgres_properties)

In [ ]:
spark.stop()